In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/csqa-logicalcombinations/test_all_hf.json
/kaggle/input/csqa-logicalcombinations/dev_all_hf.json
/kaggle/input/csqa-logicalcombinations/test_logical_combinations_output.json
/kaggle/input/csqa-logicalcombinations/dev_logical_combinations_output.json
/kaggle/input/csqa-logicalcombinations/logical_combinations_output.json
/kaggle/input/csqa-logicalcombinations/train_logical_combinations_output.json
/kaggle/input/csqa-logicalcombinations/train_all_hf.json
/kaggle/input/csqa-pairs-baselines/QA_Mixed.csv
/kaggle/input/csqa-pairs-baselines/csqa_zeroshot_NEITHER_results.csv
/kaggle/input/csqa-pairs-baselines/QA_NEITHER.csv
/kaggle/input/csqa-pairs-baselines/csqa_zeroshot_OR_results.csv
/kaggle/input/csqa-pairs-baselines/QA_AND.csv
/kaggle/input/csqa-pairs-baselines/__huggingface_repos__.json
/kaggle/input/csqa-pairs-baselines/csqa_zeroshot_MIXED_results.csv
/kaggle/input/csqa-pairs-baselines/csqa_zeroshot_AND_results.csv
/kaggle/input/csqa-pairs-baselines/QA_OR.csv
/kaggle/input

In [2]:
import json
import csv

# Read the JSONL file (one JSON object per line)
with open("/kaggle/input/csqa-logicalcombinations/test_all_hf.json", "r") as f:
    data = [json.loads(line) for line in f]

# Write to CSV
with open('test_dataset.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    
    # Write header
    writer.writerow(['question', 'A', 'B', 'C', 'D', 'correct_label', 'correct_answer_text', 'qa_type'])
    
    # Write data rows
    for item in data:
        choices = item['choices']
        label = item['label']
        correct_answer = choices[label]
        
        # Map integer label to letter
        label_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
        label_letter = label_map[label]
        
        correct_answer = choices[label]
        
        row = [
            item['question'],
            choices[0] if len(choices) > 0 else '',
            choices[1] if len(choices) > 1 else '',
            choices[2] if len(choices) > 2 else '',
            choices[3] if len(choices) > 3 else '',
            label_letter,  # Now shows A, B, C, or D
            correct_answer,
            item['qa_type']
        ]
        writer.writerow(row)

print("Conversion complete! Check test_dataset.csv")


Conversion complete! Check test_dataset.csv


In [3]:
qa_df = pd.read_csv('/kaggle/working/test_dataset.csv')
qa_df.head()

,question,A,B,C,D,correct_label,correct_answer_text,qa_type
0,What happens when applying a heat source to co...,emits toxic fumes AND decomposes into gases,burns without oxygen AND produces harmless vapors,ignites rapidly AND melts into liquid,decomposes into gases AND creates structural c...,A,emits toxic fumes AND decomposes into gases,AND
1,The man with Crohn's disease was eating a hamb...,inflammation flare-up AND increased calorie in...,inflammation flare-up AND reduced nutrient uptake,inflammation flare-up AND sudden appetite incr...,intestinal discomfort AND sudden appetite incr...,B,inflammation flare-up AND reduced nutrient uptake,AND
2,Where would you find a desk normally occupied ...,youth community workspace AND corporate office,adult classroom AND public library,study group area AND homework station,corporate office AND adult classroom,C,study group area AND homework station,AND
3,"A lonely man committed suicide, why would he d...",thought he was happy AND believed in a better ...,sought emotional escape AND assumed others wou...,sought emotional escape AND thought he was happy,felt like a burden AND sought emotional escape,D,felt like a burden AND sought emotional escape,AND
4,Where might a child go who has no home?,youth shelter AND community center,youth shelter AND school event,foster care AND youth club,foster care AND family gathering,A,youth shelter AND community center,AND


In [4]:
!pip install -q -U transformers accelerate bitsandbytes peft trl datasets 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

os.environ['HF_TOKEN'] = user_secrets.get_secret("hugging_face")

In [6]:
from huggingface_hub import login
login(new_session=False)

In [7]:
# import pandas as pd
# import random
# import ast

# # Load your dataset
# df = pd.read_csv("/kaggle/working/logical_combinations_output.csv")  # replace with your file

# # Safe eval to ensure lists are properly loaded
# def safe_eval(val):
#     if isinstance(val, list):
#         return val
#     if pd.isna(val):
#         return []
#     if isinstance(val, str):
#         try:
#             return ast.literal_eval(val)
#         except:
#             return []
#     return []

# df['logical_combinations'] = df['logical_combinations'].apply(safe_eval)

# # Prepare storage for QA sets
# qa_sets = {
#     'AND': [],
#     'OR': [],
#     'NEITHER': [],
#     'Mixed': []
# }

# # Label rotation - separate rotator for each question type
# class LabelRotator:
#     def __init__(self, name):
#         self.name = name
#         self.labels = ['A', 'B', 'C', 'D']
#         self.current_index = 0
    
#     def get_next_label(self):
#         label = self.labels[self.current_index]
#         self.current_index = (self.current_index + 1) % len(self.labels)
#         # print(f"DEBUG {self.name}: Assigned label {label}, next will be {self.labels[self.current_index]}")
#         return label

# # Create separate rotators for each question type
# rotators = {
#     'AND': LabelRotator('AND'),
#     'OR': LabelRotator('OR'),
#     'NEITHER': LabelRotator('NEITHER'),
#     'Mixed': LabelRotator('Mixed')
# }

# def create_qa_item(question, correct_ans, incorrect_ans, qa_type):
#     """Helper function to create a QA item with proper label rotation"""
    
#     # Get correct label from the appropriate rotator
#     correct_label = rotators[qa_type].get_next_label()
    
#     # Ensure we have exactly 3 incorrect answers
#     if len(incorrect_ans) < 3:
#         print(f"Warning: Only {len(incorrect_ans)} incorrect answers available for {qa_type}")
#         return None
    
#     selected_incorrect = random.sample(incorrect_ans, 3)
    
#     # Create options array with 4 positions
#     options = [''] * 4
    
#     # Place correct answer at the designated position
#     correct_pos = rotators[qa_type].labels.index(correct_label)
#     options[correct_pos] = correct_ans
    
#     # Fill remaining positions with incorrect answers
#     incorrect_positions = [i for i in range(4) if i != correct_pos]
#     for i, pos in enumerate(incorrect_positions):
#         options[pos] = selected_incorrect[i]
    
#     return {
#         'question': question,
#         'A': options[0],
#         'B': options[1],
#         'C': options[2],
#         'D': options[3],
#         'correct_label': correct_label,
#         'correct_answer_text': correct_ans,
#         'qa_type': qa_type
#     }

# # Track counts for each type
# type_counts = {'AND': 0, 'OR': 0, 'NEITHER': 0, 'Mixed': 0}

# for idx, row in df.iterrows():
#     question = row['question']
#     combos = row['logical_combinations']
    
#     # Extract AND/OR/NEITHER correct and incorrect lists
#     and_correct = combos.get('AND_combinations', {}).get('correct', [])
#     and_incorrect = combos.get('AND_combinations', {}).get('incorrect', [])
    
#     or_correct = combos.get('OR_combinations', {}).get('correct', [])
#     or_incorrect = combos.get('OR_combinations', {}).get('incorrect', [])
    
#     neither_correct = combos.get('NEITHER_combinations', {}).get('correct', [])
#     neither_incorrect = combos.get('NEITHER_combinations', {}).get('incorrect', [])
    
#     # --- AND-only QA ---
#     if and_correct and len(and_incorrect) >= 3:
#         correct_ans = random.choice(and_correct)
#         qa_item = create_qa_item(question, correct_ans, and_incorrect, 'AND')
#         if qa_item:
#             qa_sets['AND'].append(qa_item)
#             type_counts['AND'] += 1
#             # print(f"AND Question {type_counts['AND']}: Label {qa_item['correct_label']}")
    
#     # --- OR-only QA ---
#     if or_correct and len(or_incorrect) >= 3:
#         correct_ans = random.choice(or_correct)
#         qa_item = create_qa_item(question, correct_ans, or_incorrect, 'OR')
#         if qa_item:
#             qa_sets['OR'].append(qa_item)
#             type_counts['OR'] += 1
#             # print(f"OR Question {type_counts['OR']}: Label {qa_item['correct_label']}")
    
#     # --- NEITHER-only QA ---
#     if neither_correct and len(neither_incorrect) >= 3:
#         correct_ans = random.choice(neither_correct)
#         qa_item = create_qa_item(question, correct_ans, neither_incorrect, 'NEITHER')
#         if qa_item:
#             qa_sets['NEITHER'].append(qa_item)
#             type_counts['NEITHER'] += 1
#             # print(f"NEITHER Question {type_counts['NEITHER']}: Label {qa_item['correct_label']}")
    
#     # --- Mixed QA ---
#     all_correct_types = [
#         ('AND', and_correct),
#         ('OR', or_correct),
#         ('NEITHER', neither_correct)
#     ]
#     all_incorrect_types = and_incorrect + or_incorrect + neither_incorrect
    
#     # Ensure at least one correct exists and enough incorrect answers
#     valid_categories = [cat for cat, lst in all_correct_types if lst]
#     if valid_categories and len(all_incorrect_types) >= 3:
#         chosen_category = random.choice(valid_categories)
#         chosen_correct_list = dict(all_correct_types)[chosen_category]
#         correct_ans = random.choice(chosen_correct_list)
#         qa_item = create_qa_item(question, correct_ans, all_incorrect_types, 'Mixed')
#         if qa_item:
#             qa_sets['Mixed'].append(qa_item)
#             type_counts['Mixed'] += 1
#             # print(f"Mixed Question {type_counts['Mixed']}: Label {qa_item['correct_label']}")

# print(f"\n=== GENERATION COMPLETE ===")
# for qtype, count in type_counts.items():
#     print(f"{qtype}: {count} questions generated")

# # --- Convert to DataFrames and save ---
# for key, qlist in qa_sets.items():
#     if qlist:  # Only save if we have questions
#         df_out = pd.DataFrame(qlist)
#         df_out.to_csv(f'QA_{key}.csv', index=False)
#         print(f"\n{key} QA saved: {len(df_out)} questions in QA_{key}.csv")
        
#         # Show label distribution for verification
#         label_counts = df_out['correct_label'].value_counts().sort_index()
#         print(f"  Label distribution: {dict(label_counts)}")
        
#         # Show label sequence for first 8 questions to verify rotation
#         print(f"  Label sequence (first 8): {list(df_out['correct_label'].head(8))}")
#     else:
#         print(f"{key}: No valid QA pairs generated")

# # Overall statistics
# total_questions = sum(len(qlist) for qlist in qa_sets.values())
# print(f"\nFinal total questions generated: {total_questions}")

In [8]:
import torch

In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

def calculate_qa_metrics(results_df):
    """
    
    Calculate comprehensive evaluation metrics for QA results
    
    Args:
        results_df: DataFrame with 'predicted_label' and 'correct_label' columns
    
    Returns:
        dict: Dictionary containing all evaluation metrics
    """
    
    # Remove None predictions for accurate calculation
    valid_predictions = results_df.dropna(subset=['predicted_label'])
    
    if len(valid_predictions) == 0:
        print("Warning: No valid predictions found!")
        return {}
    
    y_true = valid_predictions['correct_label'].tolist()
    y_pred = valid_predictions['predicted_label'].tolist()
    
    # Basic metrics
    total_questions = len(results_df)
    answered_questions = len(valid_predictions)
    unanswered_questions = total_questions - answered_questions
    
    # Accuracy (main metric for QA)
    accuracy = accuracy_score(y_true, y_pred)
    
    # Per-class metrics
    labels = ['A', 'B', 'C', 'D']
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, labels=labels, average=None, zero_division=0
    )
    
    # Macro and micro averages
    macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro', zero_division=0
    )
    micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='micro', zero_division=0
    )
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    
    # Answer rate (percentage of questions answered)
    answer_rate = answered_questions / total_questions
    
    # Compile results
    metrics = {
        'total_questions': total_questions,
        'answered_questions': answered_questions,
        'unanswered_questions': unanswered_questions,
        'answer_rate': answer_rate,
        'accuracy': accuracy,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'macro_f1': macro_f1,
        'micro_precision': micro_precision,
        'micro_recall': micro_recall,
        'micro_f1': micro_f1,
    }
    
    # Per-class metrics
    for i, label in enumerate(labels):
        metrics[f'{label}_precision'] = precision[i]
        metrics[f'{label}_recall'] = recall[i]
        metrics[f'{label}_f1'] = f1[i]
        metrics[f'{label}_support'] = support[i]
    
    return metrics, cm, y_true, y_pred

def print_qa_metrics(metrics):
    """Pretty print the evaluation metrics"""
    print("=" * 50)
    print("QA EVALUATION METRICS")
    print("=" * 50)
    
    print(f"Total Questions: {metrics['total_questions']}")
    print(f"Answered Questions: {metrics['answered_questions']}")
    print(f"Answer Rate: {metrics['answer_rate']:.1%}")
    print(f"Accuracy: {metrics['accuracy']:.1%}")
    
    print("\n" + "-" * 30)
    print("OVERALL METRICS")
    print("-" * 30)
    print(f"Macro Precision: {metrics['macro_precision']:.3f}")
    print(f"Macro Recall: {metrics['macro_recall']:.3f}")
    print(f"Macro F1: {metrics['macro_f1']:.3f}")
    print(f"Micro Precision: {metrics['micro_precision']:.3f}")
    print(f"Micro Recall: {metrics['micro_recall']:.3f}")
    print(f"Micro F1: {metrics['micro_f1']:.3f}")
    
    print("\n" + "-" * 40)
    print("PER-CLASS METRICS")
    print("-" * 40)
    print(f"{'Class':<5} {'Precision':<9} {'Recall':<6} {'F1':<6} {'Support'}")
    print("-" * 40)
    
    for label in ['A', 'B', 'C', 'D']:
        precision = metrics[f'{label}_precision']
        recall = metrics[f'{label}_recall']
        f1 = metrics[f'{label}_f1']
        support = int(metrics[f'{label}_support'])
        print(f"{label:<5} {precision:<9.3f} {recall:<6.3f} {f1:<6.3f} {support}")

def plot_confusion_matrix(cm, labels=['A', 'B', 'C', 'D'], save_path=None):
    """Plot confusion matrix"""
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=labels, yticklabels=labels)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def analyze_answer_distribution(results_df):
    """Analyze the distribution of correct and predicted answers"""
    print("\n" + "=" * 50)
    print("ANSWER DISTRIBUTION ANALYSIS")
    print("=" * 50)
    
    # True label distribution
    true_dist = results_df['correct_label'].value_counts().sort_index()
    print("\nTrue Label Distribution:")
    for label in ['A', 'B', 'C', 'D']:
        count = true_dist.get(label, 0)
        pct = count / len(results_df) * 100
        print(f"  {label}: {count} ({pct:.1f}%)")
    
    # Predicted label distribution (excluding None)
    valid_preds = results_df.dropna(subset=['predicted_label'])
    if len(valid_preds) > 0:
        pred_dist = valid_preds['predicted_label'].value_counts().sort_index()
        print("\nPredicted Label Distribution:")
        for label in ['A', 'B', 'C', 'D']:
            count = pred_dist.get(label, 0)
            pct = count / len(valid_preds) * 100
            print(f"  {label}: {count} ({pct:.1f}%)")
    
    # Check for bias towards certain options
    print("\nLabel Rotation Check:")
    if len(set(true_dist.values)) == 1:
        print("Perfect label rotation detected")
    else:
        print("Uneven label distribution detected")
        
def comprehensive_qa_evaluation(results_df, save_plots=True):
    """Run complete evaluation pipeline"""
    
    # Calculate metrics
    metrics, cm, y_true, y_pred = calculate_qa_metrics(results_df)
    
    if not metrics:
        return
    
    # Print metrics
    print_qa_metrics(metrics)
    
    # Analyze distributions
    analyze_answer_distribution(results_df)
    
    # Plot confusion matrix
    if save_plots:
        plot_confusion_matrix(cm, save_path='confusion_matrix.png')
    else:
        plot_confusion_matrix(cm)
    
    # Print sklearn classification report for additional insights
    valid_preds = results_df.dropna(subset=['predicted_label'])
    if len(valid_preds) > 0:
        print("\n" + "=" * 50)
        print("DETAILED CLASSIFICATION REPORT")
        print("=" * 50)
        print(classification_report(
            valid_preds['correct_label'], 
            valid_preds['predicted_label'],
            labels=['A', 'B', 'C', 'D']
        ))
    
    return metrics

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


MODEL_NAME = "allenai/Olmo-3-7B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Load model in 4-bit for QLoRA
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_4bit=True,              # 4-bit quantization
    device_map="auto",              # Automatically split layers to GPU
    bnb_4bit_quant_type="nf4",      # NormalFloat4 (better for QLoRA)
    bnb_4bit_use_double_quant=True, # More compression
    torch_dtype="auto"
)

# Prepare for LoRA fine-tuning
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8,            # Rank
    lora_alpha=16,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

print("Model ready for QLoRA fine-tuning!")


2025-11-29 06:12:46.850310: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764396767.032355      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764396767.084108      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/204 [00:00<?, ?B/s]

Model ready for QLoRA fine-tuning!


In [11]:
def zero_shot_qa(model, tokenizer, qa_dataframe):
    results = []
    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain the following columns: {required_cols}")
    
    valid_choices = {'A', 'B', 'C', 'D'}
    
    for idx, row in qa_dataframe.iterrows():
        predicted_label = None
        try:

            prompt = f"""Answer the following commonsense question by selecting the correct option.
            Instructions: Please respond with ONLY the single, capital letter (A, B, C, or D) that is the correct answer. 
            Do not include any other text, punctuation, or explanation.

Question: {str(row['question'])}

A. {str(row['A'])}
B. {str(row['B'])}
C. {str(row['C'])}
D. {str(row['D'])}

Answer:"""


            conversation = [
                {"role": "system", "content": "You are a helpful assistant. Answer the following commonsense question by selecting the correct option."},
                {"role": "user", "content": prompt}
            ]

            # 1. TOKENIZE the input prompt string
            tokenized_inputs = tokenizer.apply_chat_template(
                conversation,
                add_generation_prompt=True,
                return_tensors="pt"
            )

            if isinstance(tokenized_inputs, torch.Tensor):
                inputs = {"input_ids": tokenized_inputs, "attention_mask": torch.ones_like(tokenized_inputs)}
            else:
                inputs = tokenized_inputs
            
            inputs = {k: v.to(model.device) for k, v in inputs.items()}
            
            # 2. GENERATE the response using the tokenized input
            # We explicitly pass `input_ids` and `attention_mask`
            output_tokens = model.generate(
                **inputs,
                max_new_tokens=3,  # just one letter
                do_sample=False,
                temperature=0.0,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id
            )
            
            # 3. DECODE the output tokens back into a string
            # We slice the output to get only the newly generated tokens
            generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)


            if "Answer:" in generated_text:
                generated_text = generated_text.split("Answer:")[-1]
            # print(generated_text)
            # Clean up extra prefixes or artifacts
            generated_text = generated_text.strip()  # remove spaces/newlines
            if generated_text.lower().startswith("assistant"):
                generated_text = generated_text.splitlines()[-1].strip()  # keep the last meaningful line

            # 4. Process the generated text for a strict answer
            # Strip all whitespace and common punctuation, then get the first character
            cleaned_text = generated_text.strip().upper().replace('.', '').replace(',', '').replace(' ', '')
            # print(cleaned_text)
            # Validate that the first character is a valid choice
            if cleaned_text and cleaned_text[0] in valid_choices:
                predicted_label = cleaned_text[0]

        except Exception as e:
            # Catch any errors during processing and log them
            print(f"Error processing row {idx}: {e}")
        
        results.append({
            'question': str(row['question']),
            'predicted_label': predicted_label,
            'correct_label': str(row['correct_label']).strip().upper(),
            'full_qa': f"""Question: {str(row['question'])} \n
            A. {str(row['A'])}
            B. {str(row['B'])}
            C. {str(row['C'])}
            D. {str(row['D'])}"""
        })
            
    return results

In [12]:
def n_shot_and_qa(model, tokenizer, qa_dataframe):
    results = []
    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain the following columns: {required_cols}")
    
    valid_choices = {'A', 'B', 'C', 'D'}
    
    for idx, row in qa_dataframe.iterrows():
        predicted_label = None
        try:

            prompt = f"""
                Answer the following commonsense question by selecting the correct option.
                Instructions: Please respond with ONLY the single, capital letter (A, B, C, or D) that is the correct answer. Do not include any other text, punctuation, or explanation.
                
                Examples:
                Question: Sammy wanted to go to where the people were. Where might he go?
                        A. social venues AND quiet retreats
                        B. local events AND social venues
                        C. sports arenas AND quiet retreats
                        D. sports arenas AND train platforms
                                
                        Answer: B

                
                Question: {str(row['question'])}
                A. {str(row['A'])}
                B. {str(row['B'])}
                C. {str(row['C'])}
                D. {str(row['D'])}
                Answer:"""


            conversation = [
                {"role": "system", "content": "You are a helpful assistant. Answer the following commonsense question by selecting the correct option."},
                {"role": "user", "content": prompt}
            ]

            # 1. TOKENIZE the input prompt string
            tokenized_inputs = tokenizer.apply_chat_template(
                conversation,
                add_generation_prompt=True,
                return_tensors="pt"
            )

            if isinstance(tokenized_inputs, torch.Tensor):
                inputs = {"input_ids": tokenized_inputs, "attention_mask": torch.ones_like(tokenized_inputs)}
            else:
                inputs = tokenized_inputs
            
            inputs = {k: v.to(model.device) for k, v in inputs.items()}
            
            # 2. GENERATE the response using the tokenized input
            # We explicitly pass `input_ids` and `attention_mask`
            output_tokens = model.generate(
                **inputs,
                max_new_tokens=3,  # just one letter
                do_sample=False,
                temperature=0.0,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id
            )
            
            # 3. DECODE the output tokens back into a string
            # We slice the output to get only the newly generated tokens
            generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)


            if "Answer:" in generated_text:
                generated_text = generated_text.split("Answer:")[-1]
            # print(generated_text)
            # Clean up extra prefixes or artifacts
            generated_text = generated_text.strip()  # remove spaces/newlines
            if generated_text.lower().startswith("assistant"):
                generated_text = generated_text.splitlines()[-1].strip()  # keep the last meaningful line

            # 4. Process the generated text for a strict answer
            # Strip all whitespace and common punctuation, then get the first character
            cleaned_text = generated_text.strip().upper().replace('.', '').replace(',', '').replace(' ', '')
            # print(cleaned_text)
            # Validate that the first character is a valid choice
            if cleaned_text and cleaned_text[0] in valid_choices:
                predicted_label = cleaned_text[0]

        except Exception as e:
            # Catch any errors during processing and log them
            print(f"Error processing row {idx}: {e}")
        
        results.append({
            'question': str(row['question']),
            'predicted_label': predicted_label,
            'correct_label': str(row['correct_label']).strip().upper(),
            'full_qa': f"""Question: {str(row['question'])} \n
            A. {str(row['A'])}
            B. {str(row['B'])}
            C. {str(row['C'])}
            D. {str(row['D'])}"""
        })
            
    return results

In [13]:
qa_df_AND = qa_df[qa_df.qa_type == 'AND']
results = n_shot_and_qa(model, tokenizer, qa_df_AND)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_1shot_AND_results.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)

for res in results[:25]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


({'total_questions': 500, 'answered_questions': 500, 'unanswered_questions': 0, 'answer_rate': 1.0, 'accuracy': 0.644, 'macro_precision': 0.6894858313087608, 'macro_recall': 0.644, 'macro_f1': 0.6402802550623232, 'micro_precision': 0.644, 'micro_recall': 0.644, 'micro_f1': 0.644, 'A_precision': 0.8870967741935484, 'A_recall': 0.44, 'A_f1': 0.5882352941176471, 'A_support': 125, 'B_precision': 0.7289719626168224, 'B_recall': 0.624, 'B_f1': 0.6724137931034483, 'B_support': 125, 'C_precision': 0.5722543352601156, 'C_recall': 0.792, 'C_f1': 0.6644295302013423, 'C_support': 125, 'D_precision': 0.569620253164557, 'D_recall': 0.72, 'D_f1': 0.6360424028268551, 'D_support': 125}, array([[55,  6, 34, 30],
       [ 2, 78, 18, 27],
       [ 3, 12, 99, 11],
       [ 2, 11, 22, 90]]), ['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D'

In [14]:
def n_shot_or_qa(model, tokenizer, qa_dataframe):
    results = []
    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain the following columns: {required_cols}")
    
    valid_choices = {'A', 'B', 'C', 'D'}
    
    for idx, row in qa_dataframe.iterrows():
        predicted_label = None
        try:

            prompt = f"""
                Answer the following commonsense question by selecting the correct option.
                Instructions: Please respond with ONLY the single, capital letter (A, B, C, or D) that is the correct answer. Do not include any other text, punctuation, or explanation.
                
                Examples:
                Question: Sammy wanted to go to where the people were. Where might he go?
                A. quiet retreats OR empty parks
                B. local events OR empty parks
                C. sports arenas OR empty parks
                D. train platforms OR empty parks
                
                Answer: B
                
                Now answer the following question:
                
                Question: {str(row['question'])}
                A. {str(row['A'])}
                B. {str(row['B'])}
                C. {str(row['C'])}
                D. {str(row['D'])}
                Answer:"""


            conversation = [
                {"role": "system", "content": "You are a helpful assistant. Answer the following commonsense question by selecting the correct option."},
                {"role": "user", "content": prompt}
            ]

            # 1. TOKENIZE the input prompt string
            tokenized_inputs = tokenizer.apply_chat_template(
                conversation,
                add_generation_prompt=True,
                return_tensors="pt"
            )

            if isinstance(tokenized_inputs, torch.Tensor):
                inputs = {"input_ids": tokenized_inputs, "attention_mask": torch.ones_like(tokenized_inputs)}
            else:
                inputs = tokenized_inputs
            
            inputs = {k: v.to(model.device) for k, v in inputs.items()}
            
            # 2. GENERATE the response using the tokenized input
            # We explicitly pass `input_ids` and `attention_mask`
            output_tokens = model.generate(
                **inputs,
                max_new_tokens=3,  # just one letter
                do_sample=False,
                temperature=0.0,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id
            )
            
            # 3. DECODE the output tokens back into a string
            # We slice the output to get only the newly generated tokens
            generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)


            if "Answer:" in generated_text:
                generated_text = generated_text.split("Answer:")[-1]
            # print(generated_text)
            # Clean up extra prefixes or artifacts
            generated_text = generated_text.strip()  # remove spaces/newlines
            if generated_text.lower().startswith("assistant"):
                generated_text = generated_text.splitlines()[-1].strip()  # keep the last meaningful line

            # 4. Process the generated text for a strict answer
            # Strip all whitespace and common punctuation, then get the first character
            cleaned_text = generated_text.strip().upper().replace('.', '').replace(',', '').replace(' ', '')
            # print(cleaned_text)
            # Validate that the first character is a valid choice
            if cleaned_text and cleaned_text[0] in valid_choices:
                predicted_label = cleaned_text[0]

        except Exception as e:
            # Catch any errors during processing and log them
            print(f"Error processing row {idx}: {e}")
        
        results.append({
            'question': str(row['question']),
            'predicted_label': predicted_label,
            'correct_label': str(row['correct_label']).strip().upper(),
            'full_qa': f"""Question: {str(row['question'])} \n
            A. {str(row['A'])}
            B. {str(row['B'])}
            C. {str(row['C'])}
            D. {str(row['D'])}"""
        })
            
    return results

In [15]:
qa_df_OR = qa_df[qa_df.qa_type == 'OR']
results = n_shot_or_qa(model, tokenizer, qa_df_OR)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_1shot_OR_results.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)


for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

({'total_questions': 500, 'answered_questions': 500, 'unanswered_questions': 0, 'answer_rate': 1.0, 'accuracy': 0.614, 'macro_precision': 0.6577100487356897, 'macro_recall': 0.614, 'macro_f1': 0.616926298133495, 'micro_precision': 0.614, 'micro_recall': 0.614, 'micro_f1': 0.614, 'A_precision': 0.8205128205128205, 'A_recall': 0.512, 'A_f1': 0.6305418719211823, 'A_support': 125, 'B_precision': 0.7582417582417582, 'B_recall': 0.552, 'B_f1': 0.638888888888889, 'B_support': 125, 'C_precision': 0.5432098765432098, 'C_recall': 0.704, 'C_f1': 0.6132404181184669, 'C_support': 125, 'D_precision': 0.5088757396449705, 'D_recall': 0.688, 'D_f1': 0.585034013605442, 'D_support': 125}, array([[64,  6, 28, 27],
       [ 7, 69, 20, 29],
       [ 2,  8, 88, 27],
       [ 5,  8, 26, 86]]), ['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D'

In [16]:
def n_shot_nnor_qa(model, tokenizer, qa_dataframe):
    results = []
    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain the following columns: {required_cols}")
    
    valid_choices = {'A', 'B', 'C', 'D'}
    
    for idx, row in qa_dataframe.iterrows():
        predicted_label = None
        try:

            prompt = f"""
                Answer the following commonsense question by selecting the correct option.
                Instructions: Please respond with ONLY the single, capital letter (A, B, C, or D) that is the correct answer. Do not include any other text, punctuation, or explanation.
                
                Examples:
                Question: Sammy wanted to go to where the people were. Where might he go? 
                A. NEITHER local events NOR train platforms 
                B. NEITHER sports arenas NOR train platforms 
                C. NEITHER social venues NOR sports arenas 
                D. NEITHER social venues NOR quiet retreats
                
                Answer: B

                Now answer the following question:
                
                Question: {str(row['question'])}
                A. {str(row['A'])}
                B. {str(row['B'])}
                C. {str(row['C'])}
                D. {str(row['D'])}
                Answer:"""


            conversation = [
                {"role": "system", "content": "You are a helpful assistant. Answer the following commonsense question by selecting the correct option."},
                {"role": "user", "content": prompt}
            ]

            # 1. TOKENIZE the input prompt string
            tokenized_inputs = tokenizer.apply_chat_template(
                conversation,
                add_generation_prompt=True,
                return_tensors="pt"
            )

            if isinstance(tokenized_inputs, torch.Tensor):
                inputs = {"input_ids": tokenized_inputs, "attention_mask": torch.ones_like(tokenized_inputs)}
            else:
                inputs = tokenized_inputs
            
            inputs = {k: v.to(model.device) for k, v in inputs.items()}
            
            # 2. GENERATE the response using the tokenized input
            # We explicitly pass `input_ids` and `attention_mask`
            output_tokens = model.generate(
                **inputs,
                max_new_tokens=3,  # just one letter
                do_sample=False,
                temperature=0.0,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id
            )
            
            # 3. DECODE the output tokens back into a string
            # We slice the output to get only the newly generated tokens
            generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)


            if "Answer:" in generated_text:
                generated_text = generated_text.split("Answer:")[-1]
            # print(generated_text)
            # Clean up extra prefixes or artifacts
            generated_text = generated_text.strip()  # remove spaces/newlines
            if generated_text.lower().startswith("assistant"):
                generated_text = generated_text.splitlines()[-1].strip()  # keep the last meaningful line

            # 4. Process the generated text for a strict answer
            # Strip all whitespace and common punctuation, then get the first character
            cleaned_text = generated_text.strip().upper().replace('.', '').replace(',', '').replace(' ', '')
            # print(cleaned_text)
            # Validate that the first character is a valid choice
            if cleaned_text and cleaned_text[0] in valid_choices:
                predicted_label = cleaned_text[0]

        except Exception as e:
            # Catch any errors during processing and log them
            print(f"Error processing row {idx}: {e}")
        
        results.append({
            'question': str(row['question']),
            'predicted_label': predicted_label,
            'correct_label': str(row['correct_label']).strip().upper(),
            'full_qa': f"""Question: {str(row['question'])} \n
            A. {str(row['A'])}
            B. {str(row['B'])}
            C. {str(row['C'])}
            D. {str(row['D'])}"""
        })
            
    return results

In [17]:
qa_df_NNOR = qa_df[qa_df.qa_type == 'NEITHER']
results = n_shot_nnor_qa(model, tokenizer, qa_df_NNOR)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_1shot_NEITHER_results.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)


for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

({'total_questions': 500, 'answered_questions': 500, 'unanswered_questions': 0, 'answer_rate': 1.0, 'accuracy': 0.094, 'macro_precision': 0.07279616698633422, 'macro_recall': 0.094, 'macro_f1': 0.07019202058623386, 'micro_precision': 0.094, 'micro_recall': 0.094, 'micro_f1': 0.094, 'A_precision': 0.0625, 'A_recall': 0.008, 'A_f1': 0.014184397163120567, 'A_support': 125, 'B_precision': 0.03125, 'B_recall': 0.008, 'B_f1': 0.012738853503184714, 'B_support': 125, 'C_precision': 0.10465116279069768, 'C_recall': 0.216, 'C_f1': 0.1409921671018277, 'C_support': 125, 'D_precision': 0.09278350515463918, 'D_recall': 0.144, 'D_f1': 0.11285266457680251, 'D_support': 125}, array([[ 1,  7, 64, 53],
       [ 2,  1, 73, 49],
       [10, 14, 27, 74],
       [ 3, 10, 94, 18]]), ['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B',

In [18]:
def n_shot_mixed_qa(model, tokenizer, qa_dataframe):
    results = []
    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain the following columns: {required_cols}")
    
    valid_choices = {'A', 'B', 'C', 'D'}
    
    for idx, row in qa_dataframe.iterrows():
        predicted_label = None
        try:

            prompt = f"""
                Answer the following commonsense question by selecting the correct option.
                Instructions: Please respond with ONLY the single, capital letter (A, B, C, or D) that is the correct answer. Do not include any other text, punctuation, or explanation.
                
                Examples:
                   Question: The fox walked from the city into the forest, what was it looking for? 
        
                    A. natural habitat AND shelter from disturbances
                    B. farm fields OR neighborhood pets
                    C. city park AND neighborhood pets
                    D. farm fields AND neighborhood pets
                        
                Answer: A


                Now answer the following question:
                
                Question: {str(row['question'])}
                A. {str(row['A'])}
                B. {str(row['B'])}
                C. {str(row['C'])}
                D. {str(row['D'])}
                Answer:"""


            conversation = [
                {"role": "system", "content": "You are a helpful assistant. Answer the following commonsense question by selecting the correct option."},
                {"role": "user", "content": prompt}
            ]

            # 1. TOKENIZE the input prompt string
            tokenized_inputs = tokenizer.apply_chat_template(
                conversation,
                add_generation_prompt=True,
                return_tensors="pt"
            )

            if isinstance(tokenized_inputs, torch.Tensor):
                inputs = {"input_ids": tokenized_inputs, "attention_mask": torch.ones_like(tokenized_inputs)}
            else:
                inputs = tokenized_inputs
            
            inputs = {k: v.to(model.device) for k, v in inputs.items()}
            
            # 2. GENERATE the response using the tokenized input
            # We explicitly pass `input_ids` and `attention_mask`
            output_tokens = model.generate(
                **inputs,
                max_new_tokens=3,  # just one letter
                do_sample=False,
                temperature=0.0,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id
            )
            
            # 3. DECODE the output tokens back into a string
            # We slice the output to get only the newly generated tokens
            generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)


            if "Answer:" in generated_text:
                generated_text = generated_text.split("Answer:")[-1]
            # print(generated_text)
            # Clean up extra prefixes or artifacts
            generated_text = generated_text.strip()  # remove spaces/newlines
            if generated_text.lower().startswith("assistant"):
                generated_text = generated_text.splitlines()[-1].strip()  # keep the last meaningful line

            # 4. Process the generated text for a strict answer
            # Strip all whitespace and common punctuation, then get the first character
            cleaned_text = generated_text.strip().upper().replace('.', '').replace(',', '').replace(' ', '')
            # print(cleaned_text)
            # Validate that the first character is a valid choice
            if cleaned_text and cleaned_text[0] in valid_choices:
                predicted_label = cleaned_text[0]

        except Exception as e:
            # Catch any errors during processing and log them
            print(f"Error processing row {idx}: {e}")
        
        results.append({
            'question': str(row['question']),
            'predicted_label': predicted_label,
            'correct_label': str(row['correct_label']).strip().upper(),
            'full_qa': f"""Question: {str(row['question'])} \n
            A. {str(row['A'])}
            B. {str(row['B'])}
            C. {str(row['C'])}
            D. {str(row['D'])}"""
        })
            
    return results

In [19]:
qa_df_mixed = qa_df[qa_df.qa_type == 'Mixed']
results = n_shot_mixed_qa(model, tokenizer, qa_df_mixed)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_1shot_MIXED_results.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)


for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

({'total_questions': 500, 'answered_questions': 500, 'unanswered_questions': 0, 'answer_rate': 1.0, 'accuracy': 0.466, 'macro_precision': 0.4792770972943387, 'macro_recall': 0.46599999999999997, 'macro_f1': 0.4618080274267199, 'micro_precision': 0.466, 'micro_recall': 0.466, 'micro_f1': 0.466, 'A_precision': 0.5222222222222223, 'A_recall': 0.376, 'A_f1': 0.4372093023255814, 'A_support': 125, 'B_precision': 0.5172413793103449, 'B_recall': 0.36, 'B_f1': 0.42452830188679247, 'B_support': 125, 'C_precision': 0.4114285714285714, 'C_recall': 0.576, 'C_f1': 0.48000000000000004, 'C_support': 125, 'D_precision': 0.46621621621621623, 'D_recall': 0.552, 'D_f1': 0.5054945054945056, 'D_support': 125}, array([[47, 18, 37, 23],
       [18, 45, 35, 27],
       [12, 12, 72, 29],
       [13, 12, 31, 69]]), ['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D',